<a href="https://colab.research.google.com/github/ArtyomShabunin/SMOPA-25/blob/main/lesson_8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src="https://prana-system.com/files/110/rds_color_full.png" alt="tot image" width="300"  align="center"/> &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src="https://mpei.ru/AboutUniverse/OficialInfo/Attributes/PublishingImages/logo1.jpg" alt="mpei image" width="200" align="center"/>
<img src="https://mpei.ru/Structure/Universe/tanpe/structure/tfhe/PublishingImages/tot.png" alt="tot image" width="100"  align="center"/>

---

# **Системы машинного обучения и предиктивной аналитики в тепловой и возобновляемой энергетике**  

# ***Практические занятия***


---

# Занятие №8
# Прогнозирование временных рядов методами машинного обучения
**9 апреля 2025г.**

---

**Временной ряд** — это последовательность числовых данных, упорядоченных во времени и собранных через **равные промежутки времени**.

**Примеры временных рядов:**
- Температура воздуха каждый час
- Цена на нефть каждый день
- Электрическая мощность установки каждую секунду
- Количество клиентов в магазине каждый день
- Давление в котле каждые 10 секунд

---

**Особенности временных рядов:**

1. **Время имеет значение** — порядок данных важен, в отличие от обычных таблиц.
2. **Зависимость от прошлого** — текущее значение может зависеть от предыдущих (автокорреляция).
3. **Стационарность или нестационарность** — поведение ряда может меняться со временем (например, тренд, изменение дисперсии).
4. **Сезонность** — повторяющиеся циклы (день-ночь, зима-лето и т.д.).

---

**Где используются:**

- Финансы (курсы акций, криптовалюты)
- Энергетика (нагрузка, температура, давление)
- Промышленность (сигналы с датчиков)
- Метеорология (погода, климат)
- Медицина (ЭКГ, пульс, мониторинг пациентов)
- Транспорт (потоки, GPS-координаты)

---

**Задача предсказания временных рядов** — это тип задачи машинного обучения или статистического анализа, в которой требуется спрогнозировать будущие значения некоторой переменной на основе её предыдущих наблюдений, упорядоченных во времени.

**Основные характеристики:**

1. **Временная зависимость**: значения переменной зависят от предыдущих значений. Это означает, что порядок данных важен, в отличие от многих других задач машинного обучения.
2. **Цель**: предсказать одно или несколько будущих значений временного ряда на основании истории.
3. **Примеры данных**: температура воздуха по дням, спрос на электроэнергию по часам, курсы валют, давление и температура в техпроцессе и т.д.

**Пример:**
Допустим, у нас есть данные о температуре каждый день за последние 30 дней. Мы хотим предсказать температуру на следующий день. Это и есть задача предсказания временного ряда.

**Типы задач:**
- **Одношаговое предсказание (one-step forecasting)** — предсказание значения на следующий момент времени.
- **Многопериодное предсказание (multi-step forecasting)** — предсказание значений на несколько будущих шагов.
- **Унивариантное (univariate)** — используется только один временной ряд.
- **Мультивариантное (multivariate)** — используются несколько временных рядов (например, температура, давление и влажность одновременно).

**Методы:**
- Классические: ARIMA, SARIMA, экспоненциальное сглаживание.
- Машинное обучение: решающие деревья, градиентный бустинг, SVM.
- Глубокое обучение: RNN, LSTM, GRU, Transformer.

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
from plotly_resampler import FigureResampler, FigureWidgetResampler

from plotly_resampler import register_plotly_resampler, unregister_plotly_resampler
register_plotly_resampler(mode="auto", default_n_shown_samples=10000)

## Загрузка данных и обработка данных
Набор содержит данные о почасовом производстве ветряной и солнечной электроэнергии (в МВт) во французской электросети с 2020 года.

In [ ]:
import gdown
import warnings
warnings.filterwarnings('ignore')
gdown.download('https://drive.google.com/uc?id=1NAYPaEkovk7jvaURdjI0nCi7CUMxry7W', verify=False)

df = pd.read_csv('./intermittent-renewables-production-france.csv')

In [ ]:
df.head()

In [ ]:
df = df.rename(columns={'Date and Hour' : 'DateTime'})
df['DateTime'] = df['DateTime'].str.slice(stop=-6)
df['DateTime'] = pd.to_datetime(df['DateTime'])
df = df.sort_values(ascending=True,by='DateTime')
df = df.drop(['Date','dayOfYear','dayName','monthName'],axis=1)
df = df.dropna()
df = df.set_index("DateTime")

In [ ]:
df.head()

In [ ]:
solar = df[df['Source'] == 'Solar']['Production']
wind = df[df['Source'] == 'Wind']['Production']

In [ ]:
solar, wind

In [ ]:
color_pal = sns.color_palette()
solar.plot(style='.',
          figsize=(20, 5),
          ms=3,
          color=color_pal[3],
          title='Солнечная электроэнергия')
plt.ylabel("МВт")
plt.show()

wind.plot(style='.',
          figsize=(20, 5),
          ms=3,
          color=color_pal[2],
          title='Ветряная электроэнергия')
plt.ylabel("МВт")
plt.show()

## Производство солнечной электроэнергии
### Разложение временного ряда

**Разложение временного ряда** — это метод, при котором временной ряд представляется как сумма (или произведение) нескольких более простых компонент:

1. **Тренд (`trend`)** — общее направление изменения данных с течением времени. Это может быть рост, спад или стабилизация.
2. **Сезонность (`seasonal`)** — периодические колебания, которые повторяются через равные интервалы времени (например, дни недели, месяцы, сезоны года).
3. **Остаток (`residual` или `noise`)** — всё, что не объясняется трендом и сезонностью. Это случайные, непредсказуемые флуктуации.

<img src="https://github.com/ArtyomShabunin/SMOPA-25/blob/main/imgs/trend_seasonality.png?raw=true" alt="trend_seasonality" width="800"  align="center"/>
* картинка из конспекта лекции Воронцова В.К. Методы машинного обучения. Инкрементное и онлайн обучение. ВМК МГУ 2022

- **Ряд 1** - сезонность без тренда
- **Ряд 2** - линейный тренд, аддитивная сезоность
- **Ряд 3** - линейный тренд, мультипликативная сезонность
- **Ряд 4** - экспоненциальный тренд, мультипликативная сезонность

---

**Математические модели**

Есть два основных способа описания этих компонентов:

1. **Аддитивная модель** (additive model)
Предполагает, что все компоненты **независимы друг от друга** и просто **суммируются**:

$$Y_t = T_t + S_t + R_t$$

Где:
- $ Y_t $ — наблюдаемое значение временного ряда в момент времени \( t \)
- $ T_t $ — тренд
- $ S_t $ — сезонная компонента
- $ R_t $ — остаток (шум)

**Когда использовать**: когда амплитуда сезонных колебаний **постоянна**, независимо от уровня тренда.

---

2. **Мультипликативная модель** (multiplicative model)

Предполагает, что компоненты взаимодействуют **мультипликативно**:

$$Y_t = T_t \times S_t \times R_t$$

**Когда использовать**: когда сезонные колебания **усиливаются или ослабевают** вместе с ростом тренда (например, расходы растут и колебания становятся больше по мере роста доходов компании).

---

**Что даёт разложение?**

1. **Анализ структуры ряда** — можно отдельно рассмотреть, какие сезонные эффекты присутствуют, каков общий тренд.
2. **Предобработка для прогнозирования** — если удалить сезонность и тренд, можно подавать чистые остатки на модель.
3. **Детекция аномалий** — если резидуальная компонента аномально большая, можно предположить сбой или событие.

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose

def seasonal_decompose_plotter(df: pd.DataFrame, model='additive', period=12, title='', figsize=(20, 12)):

    # period - период сезонности

    decomposition = seasonal_decompose(df.values, model=model, period=period)
    de_season = decomposition.seasonal
    de_resid = decomposition.resid
    de_trend = decomposition.trend

    fig, ax = plt.subplots(4, sharex=True, figsize=figsize)

    ax[0].set_title(title)
    ax[0].plot(df.index, df.values, color='C3')
    ax[0].set_ylabel(df.keys()[0])
    ax[0].grid(alpha=0.25)

    ax[1].plot(df.index, de_trend, color='C1')
    ax[1].set_ylabel('Trend')
    ax[1].grid(alpha=0.25)

    ax[2].plot(df.index, de_season, color='C2')
    ax[2].set_ylabel('Seasonal')
    ax[2].grid(alpha=0.25)

    ax[3].axhline(y=0, color='k', linewidth=1)
    ax[3].scatter(df.index, de_resid, color='C0', s=10)
    ax[3].set_ylabel('Resid')
    ax[3].grid(alpha=0.25)

    plt.tight_layout(h_pad=0)
    plt.show()

    return decomposition

In [ ]:
_ = seasonal_decompose_plotter(solar, model='additive', period=24*365, title='Solar Power Generation Seasonal Decompose', figsize=(20, 12))

### Предсказание производства солнечной электроэнергии

**Prophet** — это библиотека для **прогнозирования временных рядов**. Она особенно удобна для пользователей, которым **нужны точные прогнозы без глубокого погружения в статистику** или машинное обучение.

---

**Основные особенности Prophet:**

**Удобство** — прост в использовании.  
**Поддерживает тренды и сезонность** — автоматически выявляет и моделирует их.  
**Гибкость** — позволяет добавлять пользовательские праздничные дни, внешние факторы, ручные настройки.  
**Подходит для бизнес-прогнозов** — хорошо работает с ежедневными, недельными и месячными данными, в том числе с пропущенными значениями.

---

**Как работает Prophet?**

Prophet использует **аддитивную модель**:

$$y(t) = g(t) + s(t) + h(t) + \varepsilon_t$$

где:

- $ g(t) $ — **тренд** (например, линейный или с изменением наклона)
- $ s(t) $ — **сезонность** (дневная, недельная, годовая)
- $ h(t) $ — **праздники или особые события**
- $ \varepsilon_t $ — **ошибка или шум**

Но **сезонность (и праздники)** могут быть как **аддитивными**, так и **мультипликативными** по отношению к тренду.


**Аддитивная сезонность:**

$$y(t) = g(t) + s(t)$$

- Сезонность остаётся одинаковой при любом уровне тренда.
- Подходит, когда амплитуда сезонных колебаний постоянна.

**Мультипликативная сезонность:**
$$
y(t) = g(t) \times (1 + s(t))
$$

- Амплитуда сезонных колебаний **зависит от уровня тренда**.
- Подходит, если сезонные колебания становятся **больше при росте** тренда (например, увеличение спроса и сезонных всплесков одновременно).


In [ ]:
# # Сократим выборку
# solar_dy_day = solar.resample('1d').sum()

In [ ]:
solar = pd.DataFrame(solar)
solar.columns = ["y"]
solar["ds"] = solar.index

In [ ]:
solar

Настройка и обучение модели предсказания

In [ ]:
from prophet import Prophet

model_param ={
    "daily_seasonality": True, # ежедневная сезонность
    "weekly_seasonality":False, # недельная сезонность
    "yearly_seasonality":True, # годовая сезонность
    "seasonality_mode": "multiplicative", # сезонность будет мультипликативной
    # "changepoint_prior_scale" : 0.05 # управляет гибкостью модели в определении изменений тренда
}

model = Prophet(**model_param)
model.fit(solar);

Получаем прогноз

In [ ]:
future= model.make_future_dataframe(
    periods=365*24*1, # количество будущих временных шагов для прогноза
    freq='h' # частота временного ряда
)
forecast= model.predict(future)

In [ ]:
fig = model.plot(
    forecast, xlabel='Datetime(gmt)', ylabel=r'Солнечная электроэнергия', figsize=(20, 5), uncertainty=True)
plt.title('Почасовая генерация солнечной электроэнергии')
plt.show()

In [ ]:
fig = model.plot(forecast,figsize=(20, 6))
plt.xlim(pd.to_datetime(['2023-03-16', '2023-03-26']))
plt.ylim(-1500,8000)
plt.show()

```plot_components``` построит графики всех доступных составляющих

In [ ]:
fig = model.plot_components(forecast,uncertainty=True,figsize=(20, 10))
plt.show()

**Почасовая или посуточная частота данных "забивает" годовой паттерн**
- Prophet по умолчанию обучает сезонности **с одинаковым приоритетом**, и если в данных **много мелких колебаний (например, почасовые пики каждый день)**, то модель может **сконцентрироваться на ежедневной сезонности**, считая её главной.
- В результате годичная сезонность "теряется", потому что её влияние **размывается среди шума и более сильных краткосрочных эффектов**.

Особенно заметно при **почасовых данных**, где на каждый год приходится **8760 точек**, и подавляющее большинство из них отражают суточные циклы.



In [ ]:
model = Prophet( daily_seasonality=False,
                 weekly_seasonality=False,
                 yearly_seasonality=True,
                 seasonality_mode='multiplicative',
                 changepoint_prior_scale=0.5,
                 holidays_prior_scale=0.1
               )

model.fit(solar)

future = model.make_future_dataframe(periods=365*24*2,freq='h')
forecast = model.predict(future)
fig = model.plot(forecast,figsize=(20, 6))
plt.show()

In [ ]:
fig = model.plot(forecast,figsize=(20, 6))
plt.xlim(pd.to_datetime(['2023-03-16', '2023-03-26']))
plt.ylim(-1500,8000)
plt.show()

In [ ]:
fig = model.plot_components(forecast,uncertainty=True,figsize=(20, 10))
plt.show()

## Производство ветряной электроэнергии
### Разложение временного ряда

In [ ]:
_ = seasonal_decompose_plotter(wind, period=365*24, title='Wind Power Generation Seasonal Decompose', figsize=(20, 12))

### Предсказание производства ветряной электроэнергии

In [ ]:
wind = pd.DataFrame(wind)
wind.columns = ["y"]
wind["ds"] = wind.index

In [ ]:
model_param ={
    "daily_seasonality": False,
    "weekly_seasonality":False,
    "yearly_seasonality":True,
    "seasonality_mode": "multiplicative",
    # "changepoint_prior_scale" : 0.5
}

model = Prophet(**model_param)
model.fit(wind)

# Create future dataframe
future= model.make_future_dataframe(periods=365*24*2 ,freq='h')
forecast= model.predict(future)

In [ ]:
fig = model.plot(forecast, xlabel='Datetime(gmt)', ylabel=r'Ветряная электроэнергия', figsize=(20, 5))
plt.title('Почасовая генерация ветряной электроэнергии')
plt.show()

In [ ]:
fig = model.plot_components(forecast,uncertainty=True,figsize=(20, 10))
plt.show()